# Random forest interpretation

how the black box of a random forest can be opened up by tracking decision paths along the trees and computing feature contributions. This way, any prediction can be decomposed into contributions from features, such that

prediction = bias + feature1contribution + ... + featurencontribution.

## treeinterpreter

In [10]:
from treeinterpreter import treeinterpreter as ti
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [11]:
from sklearn.datasets import load_boston
boston = load_boston()
rf = RandomForestRegressor()
rf.fit(boston.data[:300], boston.target[:300])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [39]:
instances_1 = boston.data[[300]]
instances_2 = boston.data[[309]]
print ("Instance 0 prediction:", rf.predict(instances_1))
print ("Instance 1 prediction:", rf.predict(instances_2))

Instance 0 prediction: [29.333]
Instance 1 prediction: [22.45]


We can now decompose the predictions into the bias term (which is just the trainset mean) and individual feature contributions, so we see which features contributed to the difference and by how much.

In [40]:
prediction, bias, contributions = ti.predict(rf, instances_1)

In [44]:
for i in range(len(instances_1)):
    print ("Instance", i)
    print ("Bias (trainset mean)", bias[i])
    print ("Feature contributions:")
    for c, feature in sorted(zip(contributions[i], 
                                 boston.feature_names), 
                             key=lambda x: -abs(x[0])):
        print (feature, round(c, 2))
    print ("-"*20)

Instance 0
Bias (trainset mean) 25.468203333333335
Feature contributions:
RM 3.71
LSTAT 0.77
TAX -0.69
PTRATIO 0.48
DIS -0.25
NOX -0.24
CRIM -0.13
ZN 0.1
B -0.09
RAD 0.09
INDUS 0.08
AGE 0.04
CHAS 0.01
--------------------


In [46]:
print (prediction)
print (bias + np.sum(contributions, axis=1))

[[29.333]]
[29.333]


# Comparing too datasets

One use case where this approach can be very useful is when comparing two datasets. For example

   Understanding the exact reasons why estimated values are different on two datasets, for example what contributes to estimated house prices being different in two neighborhoods.
   
   Debugging models and/or data, for example understanding why average predicted values on newer data do not match the results seen on older data.